In [2]:
import copy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Button, Slider
from scipy.interpolate import interp1d
from tools.schemes import  K_gen, Solver, Sweeper, get_init

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.95
plt.rcParams['figure.subplot.bottom'] = 0.1
plt.rcParams['figure.subplot.hspace'] = 0.3

In [3]:
%matplotlib qt
custom_plots ={}

In [36]:
real_data = pd.read_excel(r'D:\Works\Diplom-work\Experiments\Kinetics\results\dmeg.xlsx')
cM=4.1 # {'ocm': 2.8, 'dmeg': 4.1, 'peta': 3}
cSol= 2.7 # {'ocm': 4, 'dmeg': 2.7, 'peta': 3}

## Initiatiation


In [28]:
cQ = 0.005
cN = 0.25
I0 = 1
I_const =0.0717608

In [6]:
class Kif(K_gen):
    l = I_const * I0
    l_ = 1e5  # 5 6

    qE = 1e9  # 8 10
    qE_ = 1e8  # 7 9
    H = 1e9  # 8 10
    diff = 1e8  # 8 9

    qH = 1e6  # 2000
    dQ = 1e9
    redQ = 1e3
    qQD = 2

    r = 1e8
    p = 1e-4
    rD_rec = 1e9
    rD_dis = 1e9
    D = 1
    D_ = 0.05

    qPh = 1e-4


def init_full_s(t, C, k=Kif()):
    [Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD] = C

    R1 = k.l * Q
    R2 = k.l_ * Qt
    R3 = k.diff * Qt * DH
    R4 = k.qE * Q_DH
    R5 = k.qE_ * Q_DHi
    # R6 = k.diff * Q_DHi
    R7 = k.H * Q_DHi
    R8 = k.diff * Q_DHb
    R9 = k.qH * Qt * QHH
    R10 = k.dQ * QH * QH
    R11 = k.redQ * Q * QHH
    R12 = k.qQD * Qt * QHD
    R13 = k.r * QH * D
    R14 = k.p * QHD
    R15 = k.rD_rec * D * D
    R16 = k.rD_dis * D * D
    R17 = k.D * Q * D
    R18 = k.D_ * QD
    R19 = k.qPh * Qt

    res = dict(
        dCl=-R1 + R2 + R10 - R11 - R17 + R18,
        dC2=R1 - R2 - R3 - R9 - R12 - R19,
        dC3=-R3 + R16,
        dC4=R3 - R4 + R5,
        dC5=R4 - R5 - R7,  # - R6
        dC7=R7 - R8,
        dC8=R8 + 2 * R9 - 2 * R10 + 2 * R11 + R12 - R13,
        dC9=R8 - R13 - 2 * R15 - 2 * R16 - R17 + R18,
        dClO=-R9 + R10 - R11 + R14,
        dCll=-R12 + R13 - R14,
        dC12=R12 + R17 - R18,
    )

    return list(res.values())


init_full_i = get_init(
    '[Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD]',
    {'Q': cQ, 'DH': cN},
)

In [7]:
class Kib(K_gen):
    l = Kif.l
    diff = Kif.diff  # 8 9
    dQ = Kif.dQ
    r = Kif.r
    p = Kif.p
    rD_rec = Kif.rD_rec
    rD_dis = Kif.rD_dis


def init_base_s(t, C, k=Kib()):
    [Q, Qt, DH, QH, D, QHD] = C

    R1 = k.l * Q
    R2 = k.diff * Qt * DH
    R3 = k.r * QH * D
    R4 = k.p * QHD
    R5 = k.dQ * QH * QH
    R6 = k.rD_rec * D * D
    R7 = k.rD_dis * D * D
    res = dict(
        dCl=-R1 + R5,
        dC2=R1 - R2,
        dC3=-R2 + R7,
        dC4=R2 - R3 - 2 * R5,
        dC5=R2 - R3 - 2 * R6 - 2 * R7,
        dC6=R3 - R4,
    )

    return list(res.values())


init_base_i = get_init(
    '[Q, Qt, DH, QH, D, QHD]',
    {'Q': cQ, 'DH': cN},
)

## Polymerization only


In [8]:
class Kpf(K_gen):
    init = 1e3
    prop = 1e3  # 2 4
    trans_sol = 5
    trans_m = 1e-2  # -3 0
    inh = 1e3  # 2 3
    ter_lin = 1e7*0
    ter_rec = 1e7
    ter_dis = 1e7


def pol_full_s(t, C, k=Kpf()):
    [D, M, PR, MR, Sol, Z] = C
    R1 = k.init * D * M
    R2 = k.prop * PR * M
    R3 = k.prop * MR * M
    R4 = k.trans_sol * PR * Sol
    R5 = k.trans_m * PR * M
    R6 = k.inh * PR * Z
    R7 = k.ter_lin * PR
    R8 = k.ter_rec * PR * PR
    R9 = k.ter_dis * PR * PR

    res = dict(
        D=-R1 + 0.0001,
        M=-R1 - R2 - R3 - R5,
        PR=R1 + R3 - R4 - R5 - R6 - R7 - 2 * R8 - 2 * R9,
        MR=-R3 + R5,
        Sol=-R4,
        Z=-R6,
    )

    return list(res.values())


pol_full_i = get_init(
    '[D, M, PR, MR, Sol, Q]',
    {'M': cM, 'Z': cQ},
)

In [9]:
class Kpb(K_gen):
    init = Kpf.init
    prop = Kpf.prop  # 2 4
    inh = Kpf.inh  # 2 3
    ter_lin = Kpf.ter_lin
    ter_rec = Kpf.ter_rec


def pol_base_s(t, C, k=Kpb()):
    [D, M, PR, Z] = C
    R1 = k.init * D * M
    R2 = k.prop * PR * M
    # R3 = k.trans_sol * PR * Sol
    # R4 = k.trans_m * PR * M
    R5 = k.inh * PR * Z
    R6 = k.ter_lin * PR
    R7 = k.ter_rec * PR * PR

    res = dict(
        D=-R1 + 0.0001,
        M=-R1 - R2,
        PR=R1 - R5 - R6 - 2 * R7,
        Z=-R5,
    )
    return list(res.values())


pol_base_i = get_init(
    '[D, M, PR, Q]',
    {'M': cM, 'Z': cQ},
)

## Polymerization + initiation

In [10]:
# class K_mix_full(K_gen):
#     # l = 1e9 * 0.00001  # 8 10
#     l= (1 - np.exp(-epsilon * L * cQ)) * KL * I0
#     l_ = 1e5  # 5 6

#     qE = 1e9  # 8 10
#     qE_ = 1e8  # 7 9
#     H = 1e9  # 8 10
#     diff = 1e8  # 8 9

#     qH = 1e6  # 2000
#     dQ = 1e9
#     redQ = 1e3
#     qQD = 2

#     r = 1e8
#     p = 1e-4 * 1e5
#     rD_rec = 1e9
#     rD_dis = 1e9
#     D = 1
#     D_ = 0.05

#     qPh = 1e-4

#     init = 1e3
#     prop = 1e3  # 2 4
#     trans_sol = 5
#     trans_m = 1e-3  # -3 0
#     inh = 1e2  # 2 3
#     ter_lin = 1e7
#     ter_rec = 1e7
#     ter_dis = 1e7


# def mix_full_s(t, C, k=K_mix_full()):
#     [Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD, M, PR, MR, Sol] = C

#     R1 = k.l * Q
#     R2 = k.l_ * Qt
#     R3 = k.diff * Qt * DH
#     R4 = k.qE * Q_DH
#     R5 = k.qE_ * Q_DHi
#     R6 = k.diff * Q_DHi
#     R7 = k.H * Q_DHi
#     R8 = k.diff * Q_DHb
#     R9 = k.qH * Qt * QHH
#     R10 = k.dQ * QH * QH
#     R11 = k.redQ * Q * QHH
#     R12 = k.qQD * Qt * QHD
#     R13 = k.r * QH * D
#     R14 = k.p * QHD
#     R15 = k.rD_rec * D * D
#     R16 = k.rD_dis * D * D
#     R17 = k.D * Q * D
#     R18 = k.D_ * QD
#     R19 = k.qPh * Qt

#     R1P = k.init * D * M
#     R2P = k.prop * PR * M
#     R3P = k.prop * MR * M
#     R4P = k.trans_sol * PR * Sol
#     R5P = k.trans_m * PR * M
#     R6P = k.inh * PR * Q
#     R7P = k.ter_lin * PR
#     R8P = k.ter_rec * PR * PR
#     R9P = k.ter_dis * PR * PR

#     res = dict(
#         Q=-R1 + R2 + R10 - R11 - R17 + R18 - R6P,
#         Qt=R1 - R2 - R3 - R9 - R12 - R19,
#         DH=-R3 + R16,
#         Q_DH=R3 - R4 + R5,
#         Q_DHi=R4 - R5 - R6 - R7,
#         Q_DHb=R7 - R8,
#         QH=R8 + 2 * R9 - 2 * R10 + 2 * R11 + R12 - R13,
#         D=R8 - R13 - 2 * R15 - 2 * R16 - R17 + R18 - R1P,
#         QHH=-R9 + R10 - R11 + R14,
#         QHD=-R12 + R13 - R14,
#         QD=R12 + R17 - R18,
#         M=-R1P - R2P - R3P,
#         PR=R1P + R3P - R4P - R5P - R6P - R7P - 2 * R8P - 2 * R9P,
#         MR=-R3P + R5P,
#         Sol=-R4P - R5P,
#     )

#     return list(res.values())


# mix_full_i = get_init(
#     '[Q, Qt, DH, Q_DH, Q_DHi, Q_DHb, QH, D, QHH, QHD, QD, M, PR, MR, Sol]',
#     {'Q': cQ, 'DH': 0.001, 'M': 2},
# )

In [57]:
class K_mix_base(K_gen):
    # # INITIATION
    # l = Kif.l
    # diff = Kif.diff
    # dQ = Kif.dQ

    # p = Kif.p
    # r = Kif.r
    # rD_rec = Kif.rD_rec
    # rD_dis = Kif.rD_dis

    # # POLYMERIZATION
    # prop = 420
    # trans_sol = Kpf.trans_sol
    # trans_m = Kpf.trans_m  # -3 0
    # inh = Kpf.inh  # 2 3
    # ter_lin = Kpf.ter_lin * 0
    # ter_rec = Kpf.ter_rec

    # INITIATION
    l = Kif.l*(10**(-1.12))
    diff = Kif.diff
    dQ = 1e7

    p = Kif.p
    r = 1.1e9
    rD_rec = Kif.rD_rec
    rD_dis = Kif.rD_dis

    # POLYMERIZATION
    prop = 290
    trans_sol = 1.8
    trans_m = 0.02
    inh = 1e5
    ter_lin =  0
    ter_rec = 1.2e6


def mix_base_s(t, C, k=K_mix_base()):
    [Q, Qt, DH, QH, D, QHD, M, PR, Sol, Z] = C

    R1 = k.l * Q
    R2 = k.diff * Qt * DH
    R3 = k.r * QH * D
    R4 = k.p * QHD
    R5 = k.dQ * QH * QH
    R6 = k.rD_rec * D * D
    R7 = k.rD_dis * D * D
    R8 = k.prop * D * M
    R9 = k.prop * PR * M
    R10 = k.trans_sol * PR * Sol
    Rll = k.trans_m * PR * M
    R12 = k.inh * PR * Z
    R122 = k.inh * PR * Q*0
    R13 = k.ter_lin * PR
    R14 = k.ter_rec * PR * PR

    res = dict(
        Q=-R1 + R5 - R122,
        Qt=R1 - R2,
        DH=-R2 + R7,
        QH=R2 - R3 - 2 * R5 + R12 ,
        D=R2 - R3 - 2 * R6 - 2 * R7 - R8,
        QHD=R3 - R4,
        M=-R8 - R9 - Rll,
        PR=R8 - R12 - R13 - 2 * R14 - R10 - R122,
        Sol=-R10,
        Z=-R12,
    )
    return list(res.values())

# Examples

In [65]:
t = np.linspace(0, 400, 10000)

In [ ]:
# Initiation full
Y = Solver(init_full_s, Kif(), init_full_i, t)
# Y = Sweeper(Solver(full_s, K(), full_i,  t),Solver(base_s, K(), base_i,  t))

LIM = None
AUTO = False

compounds = dict(
    Q=0,
    # DH=0,
    QHH=2,
    QH=2,
    D=1,
    QHD=2,
)
custom_plots = {}

In [ ]:
# Initiation base
# Y = Solver(init_base_s, Kib(), init_base_i, t)
Y = Sweeper(
    Solver(init_full_s, Kif(), init_full_i, t),
    Solver(init_base_s, Kib(), init_base_i, t),
)

LIM = None
AUTO = False
# compounds = dict(
#     Q=0,
#     D=1,
#     Qt=1,
#     QH=1,
#     DH=2,
# )

compounds = dict(
    Q=1,
    Qt=1,
    D=0,
    DH=2,
)
custom_plots = {}

In [ ]:
# Polimerization full
Y = Solver(pol_full_s, Kpf(), pol_full_i, t)
LIM = None
AUTO = False


compounds = dict(
    # Q=1,
    M=0,
    # Sol=1,
    MR=1,
    D=1,
)


def a_plot():
    return (1 - Y['M'] / Y.initial['M']) * 100


custom_plots = dict(
    conv=a_plot,
)

In [ ]:
# Polimerization base
# Y = Solver(pol_base_s, Kpb(), pol_base_i, t)
Y = Sweeper(
    Solver(pol_full_s, Kpf(), pol_full_i, t),
    Solver(pol_base_s, Kpb(), pol_base_i, t),
)

LIM = None
AUTO = False

compounds = dict(
    M=0,
    D=1,
    PR=2,
)


def a_plot():
    return (1 - Y['M'] / Y.initial['M']) * 100


def a_dif_plot():
    return (1 - Y.y1['M'] / Y.y1.initial['M']) * 100 - (1 - Y.y2['M'] / Y.y2.initial['M']) * 100


custom_plots = dict(
    # conv=a_plot,
)

In [66]:
# Mix base
mix_base_i = get_init(
    '[Q, Qt, DH, QH, D, QHD, M, PR, Sol, Z]',
    {'Q': cQ, 'DH': cN, 'M': cM, 'Sol': cSol, 'Z': 0.002},
)
Y = Solver(mix_base_s, K_mix_base(), mix_base_i, t)
Y2 = Solver(mix_base_s, K_mix_base(), mix_base_i, t)
Y_static = Solver(mix_base_s, K_mix_base(), mix_base_i, t)
LIM = None
AUTO = False


def a_plot():
    return (1 - Y['M'] / Y.initial['M']) * 100


def simple_p():
    return (1 - np.exp(-((t / 10) ** 4))) * 100

def real_plot():
    interpol= interp1d(real_data['t'], real_data['p'].apply( lambda x: x if x>0 else x*0.01 ),  kind='linear',fill_value='extrapolate')
    return interpol(t)*100

# def a_dif_plot():
#     return (1 - Y.y1['M'] / Y.y1.initial['M']) * 100 - (1 - Y.y2['M'] / Y.y2.initial['M']) * 100

custom_plots = dict(
    conv=a_plot,
    # simple=simple_p,
    real= real_plot,
)
compounds = dict(
    Q=2,
    D=1,
    PR=1,
)

# Plots

In [61]:
# Plots
fig, ax = plt.subplots()
gs = plt.GridSpec(2, 2, figure=fig)
fig.subplots_adjust(left=0.30, right=0.99, bottom=0.05, top=0.95, hspace=0.1, wspace=0.1)
ax.xaxis.set_ticklabels([], minor=False)
ax.yaxis.set_ticklabels([], minor=False)


ax.xaxis.set_tick_params(size=0)
ax.yaxis.set_tick_params(size=0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
# scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
scal = FuncFormatter(lambda x, pos: f'{x: .0f}')
K_stricted = Y.K


def get_ax(gs, keys_num, plots):
    ax = plt.subplot(gs)

    for c in [key for key, value in compounds.items() if value == keys_num]:
        plots[c] = ax.plot(Y.T, Y[c], label=c)[0]
    if keys_num == 0:
        for c, func_plot in custom_plots.items():
            plots[c] = ax.plot(
                Y.T,
                func_plot(),
                label=c,
            )[0]

    ax.legend()
    ax.xaxis.set_major_formatter(scal)
    if LIM is not None:
        ax.set_ylim([-LIM, LIM])

    return ax


# Plots
plots = {}
ax0 = get_ax(gs=gs[0, 0:2], keys_num=0, plots=plots)
ax1 = get_ax(gs=gs[1, 0], keys_num=1, plots=plots)
ax2 = get_ax(gs=gs[1, 1], keys_num=2, plots=plots)

# Sliders
sliders = {}
buttons = {}
K_new = K_stricted.copy()


def resolve(event=None):
    Y.solve(K=K_new)
    for c, plot in plots.items():
        if c in custom_plots:
            plot.set_ydata(custom_plots[c]()),
            continue
        plot.set_ydata(Y[c])
    fig.canvas.draw_idle()


def get_slider_action(k):
    def update(val):
        K_new[k] = K_stricted[k] * 10**val
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color = 'white'
        if AUTO:
            resolve()

    return update


def get_zero_button_action(k):
    def update(event):
        K_new[k] = 0
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color = 'red'
        fig.canvas.draw_idle()
        if AUTO:
            resolve()

    return update


def reset(event):
    global K_new
    K_new = copy.deepcopy(K_stricted)
    for k in sliders:
        sliders[k].reset()
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color = 'white'
    fig.canvas.draw_idle()
    if AUTO:
        resolve()


solve_axes = fig.add_axes([0.02, 0.05, 0.05, 0.05])
solve_button = Button(solve_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(resolve)

reset_axes = fig.add_axes([0.08, 0.05, 0.05, 0.05])
reset_button = Button(reset_axes, 'Reset', hovercolor='0.975')
reset_button.on_clicked(reset)

for i, k in enumerate(K_stricted):
    k_axes = fig.add_axes(
        [
            0.06,  # left
            0.95 - 0.04 * i,  # bottom
            0.08,  # width
            0.03,  # height
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-2,
        valmax=2,
        valinit=0,
        valstep=0.02,
        orientation="horizontal",
    )
    sliders[k] = amp_slider
    amp_slider.on_changed(get_slider_action(k))

    k_zero_axes = fig.add_axes(
        [
            0.19,  # left
            0.955 - 0.04 * i,  # bottom
            0.05,  # width
            0.02,  # height
        ]
    )
    zero_button = Button(k_zero_axes, f"{K_new[k]: .1e}", color='white')
    buttons[k] = zero_button
    zero_button.on_clicked(get_zero_button_action(k))

In [68]:
# 2 Plots
fig, ax = plt.subplots()
fig.subplots_adjust(left=0.25, right=0.99, bottom=0.05, top=0.95, hspace=0.1, wspace=0.1)
scal = FuncFormatter(lambda x, pos: f'{x: .0f}')
K_stricted = Y.K
ax.set_ylabel('Conversion [%]')
ax.set_xlabel('Time [s]')

# Plots
static_plot = ax.plot(Y_static.T, (1 - Y_static['M'] / Y_static.initial['M']) * 100, label='K x1')[0]
dynamic_plot = ax.plot(Y.T,(1 - Y['M'] / Y.initial['M']) * 100,label='K x10')[0]
dynamic_plot2 = ax.plot(Y.T,(1 - Y2['M'] / Y2.initial['M']) * 100,label='K x0.1')[0]
ax.legend()
ax.xaxis.set_major_formatter(scal)
if LIM is not None:
    ax.set_ylim([-LIM, LIM])

# Sliders
sliders = {}
buttons = {}
K_new = K_stricted.copy()
K_new2 = K_stricted.copy()


def resolve(event=None):
    Y.solve(K=K_new)
    Y2.solve(K=K_new2)
    dynamic_plot.set_ydata((1 - Y['M'] / Y.initial['M']) * 100)
    dynamic_plot2.set_ydata((1 - Y2['M'] / Y2.initial['M']) * 100)
    fig.canvas.draw_idle()


def get_slider_action(k):
    def update(val):
        K_new[k] = K_stricted[k] * 10**val
        K_new2[k] = K_stricted[k] * 10**(-val)
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color='white'
        if AUTO:
            resolve()

    return update


def get_zero_button_action(k):
    def update(event):
        K_new[k] = 0
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color='red'
        fig.canvas.draw_idle()
        if AUTO:
            resolve()


    return update


def reset(event):
    global K_new
    K_new = copy.deepcopy(K_stricted)
    for k in sliders:
        sliders[k].reset()
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        buttons[k].color='white'
    fig.canvas.draw_idle()
    if AUTO:
        resolve()


solve_axes = fig.add_axes([0.02, 0.05, 0.05, 0.05])
solve_button = Button(solve_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(resolve)

reset_axes = fig.add_axes([0.08, 0.05, 0.05, 0.05])
reset_button = Button(reset_axes, 'Reset', hovercolor='0.975')
reset_button.on_clicked(reset)

for i, k in enumerate(K_stricted):
    k_axes = fig.add_axes(
        [
            0.03,  # left
            0.95 - 0.04 * i,  # bottom
            0.10,  # width
            0.03,  # height
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-3,
        valmax=3,
        valinit=0,
        valstep=0.1,
        orientation="horizontal",
    )
    sliders[k] = amp_slider
    amp_slider.on_changed(get_slider_action(k))

    k_zero_axes = fig.add_axes(
        [
            0.15,  # left
            0.955 - 0.04 * i,  # bottom
            0.05,  # width
            0.02,  # height
        ]
    )
    zero_button = Button(k_zero_axes, f"{K_new[k]: .1e}",color='white')
    buttons[k] = zero_button
    zero_button.on_clicked(get_zero_button_action(k))

In [ ]:
Y1 = Solver(init_full_s, Kif(), init_full_i, t)
Y2 = Solver(init_base_s, Kib(), init_base_i, t)

In [ ]:
plt.plot(t,Y1['D'])
plt.plot(t,Y2['D'])

# View

In [60]:
# View
K_standart = Y.K.copy()
def find_t():
    f=(1 - Y['M'] / Y.initial['M']) * 100
    g1 = np.gradient(f,t)
    g2 = np.gradient(g1,t)
    return t[g2==g2[5:].max()][0],g1.max()
data = []
for key, value in K_standart.items():
    for i in [0.1, 1, 10]:
        k = K_standart.copy()
        k[key] = value * i
        Y.solve(K=k)
        t_max, vel = find_t()
        data.append(dict(key=key, i=i, t_max=t_max, vel=vel))
    print(key)
a= pd.DataFrame(data)
a.head()
b= a.pivot(
    index='key',
    columns='i',
    values=['t_max','vel']
)
b.to_excel('Kinetic_results.xlsx')

l
diff
dQ
p
r
rD_rec
rD_dis
prop
trans_sol
trans_m
inh
ter_lin
ter_rec


In [ ]:
# Real
def real_plot():
    interpol= interp1d(real_data['t'], real_data['p'].apply( lambda x: x if x>0 else x*0.01 ),  kind='linear',fill_value='extrapolate')
    return interpol(t)*100
plt.plot(t,real_plot(), c='orange')

# P

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, (1 - np.exp(-((t / 10) ** 4))) * 100,  label='Exposition experiment')

def old_a(t):
    cR = 1e-7
    r_light = Kif.l * cQ
    inh = Kpf.inh
    ter_lin = Kpf.ter_lin
    inh2 = Kpf.ter_dis
    gamma = 4

    t_inh = 1 / (inh * r_light) if inh != 0 else np.inf
    t_p = (ter_lin + inh2 * cR) / (Kpf.prop * r_light)
    d = inh * cQ / (ter_lin + inh2 * cR)
    print(f'{r_light=:.2}\n{d=:.2}\n{t_inh=:.2}\n{t_p=:.2}')

    return (1 - np.exp(-((t / (1 + d) / t_p) ** gamma))) * 100
    # return 1 - np.exp(-(t_inh / t_p * np.log((d + np.exp(t / t_inh)) / (d + 1))) ** gamma)

ax.plot(t, old_a(t), label='Old model')

# def new_a(t):
#     cR = 1e-7
#     return (
#         1
#         - np.exp(
#             -((t * Kpf.prop * Kif.l *cQ / (Kpf.ter_lin + Kpf.inh * cQ + Kpf.ter_rec * cR)) ** 4)
#         )
#     ) * 100


# ax.plot(t, a_plot(), label='Solved')
# ax.plot(t, new_a(t), label='New')

def old_Q(t):
    return cQ * np.exp(-Kif.l * t)
ax2 = ax.twinx()
ax2.plot(t, old_Q(t), c='black',label='Q',linestyle='dashed')


ax.legend()
ax2.legend()

# H0 solving

In [ ]:
def get_H0(Y: Solver):
    P_max = 80
    a = lambda Y: (1 - Y['M'] / Y.initial['M']) * 100
    interpol = np.poly1d(np.polyfit(a(Y), Y.T, 2))
    time_x = interpol(2)

    return time_x / (np.log(P_max / (P_max - 2)) ** (1 / 4))*I0

In [ ]:
get_H0(Y)